*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train = pd.read_csv(train_file_path, sep='\t', names=['Label', 'SMS'])
test = pd.read_csv(test_file_path, sep='\t', names=['Label', 'SMS'])

In [ ]:
train_data = train.copy()
train_data['SMS'] = train_data['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()
train_data['SMS'] = train_data['SMS'].str.lower()
train_data['SMS'] = train_data['SMS'].str.split()

test_data = test.copy()
test_data['SMS'] = test_data['SMS'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()
test_data['SMS'] = test_data['SMS'].str.lower()
test_data['SMS'] = test_data['SMS'].str.split()

In [ ]:
train_data['Spam'] = train_data['Label'].apply({'ham':0, 'spam':1}.get) 
test_data['Spam'] = test_data['Label'].apply({'ham':0, 'spam':1}.get) 

In [ ]:
# Defining pre-processing hyperparameters
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_data['SMS'])

In [ ]:
len(tokenizer.word_index)

In [ ]:
# Sequencing and padding on training and testing 
training_sequences = tokenizer.texts_to_sequences(train_data['SMS'])
training_padded = pad_sequences(training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )
testing_sequences = tokenizer.texts_to_sequences(test_data['SMS'])
testing_padded = pad_sequences(testing_sequences, maxlen = max_len,
padding = padding_type, truncating = trunc_type)

In [ ]:
training_padded.shape, testing_padded.shape

In [ ]:
training_padded[1]

In [ ]:
vocab_size = 500 # As defined earlier
embeding_dim = 16
drop_value = 0.2 # dropout
n_dense = 24

In [ ]:
#Dense model architecture
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embeding_dim, input_length=max_len))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(24, activation='relu'))
model.add(tf.keras.layers.Dropout(drop_value))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'AUC'])

In [ ]:
# fitting a dense spam detector model
num_epochs = 30
#early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(training_padded, train_data['Spam'], epochs=num_epochs, validation_data=(testing_padded, test_data['Spam']), verbose=2)

In [ ]:
model.evaluate(testing_padded, test_data.Spam)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  text_sequences = tokenizer.texts_to_sequences([pred_text])
  text_padded = pad_sequences(text_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )
  pred = model.predict([text_padded], verbose=0)
  prediction = [pred[0][0], 'ham']
  if pred >= .2:
    prediction[1] = 'spam'

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
predict_message("sale today! to stop texts call 98912460324")

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
